In [ ]:
#면류 학습

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Dense
from tensorflow.keras.regularizers import l2
import os
import numpy as np
import xml.etree.ElementTree as ET
import functools
from tensorflow.keras.optimizers import RMSprop

'''
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
'''

# 데이터증강기법
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    preprocessing_function=lambda img: tf.image.random_crop(img, [256, 256, 3])
)

# 훈련 데이터셋
train_directory = 'E:/종설/Training/1/[원천]면류'

train_dataset = train_datagen.flow_from_directory(
    train_directory,
    target_size=(256, 256),
    batch_size=24,
    class_mode='categorical'
)

# 크기조정만
val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1.0 / 255
)

# 검증데이터
val_directory = 'E:/종설/Validation/1/[원천]면류'

val_dataset = val_datagen.flow_from_directory(
    val_directory,
    target_size=(256, 256),
    batch_size=24,
    class_mode='categorical'
)

physical_devices = tf.config.list_physical_devices('CPU')
tf.config.set_visible_devices(physical_devices, 'CPU')


# EfficientNet-B0모델 불러오기
pretrained_model = hub.KerasLayer("https://tfhub.dev/google/efficientnet/b0/feature-vector/1", trainable=False)

# 층 추가
model = Sequential()
model.add(pretrained_model)
model.add(Dropout(0.3))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.3))

#매번바꿔줘야함 -이거-
model.add(Dense(train_dataset.num_classes, activation='softmax', kernel_regularizer=l2(0.0005)))

# 컴파일 - 학습률조정
model.compile(
    optimizer=RMSprop(lr=0.0005),  # Use RMSprop as the optimizer
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

num_epochs = 50
best_val_acc = 0.0


history = model.fit(
    train_dataset,
    epochs=num_epochs,
    validation_data=val_dataset,
)

# Save the model
best_val_acc = max(history.history['val_accuracy'])
model.save('my_model_noodle_numpy_v1.h5')